In [57]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from tqdm import tqdm
import re

In [58]:
raw_datasets = load_dataset("lc_quad")

Using custom data configuration default
Reusing dataset lc_quad (/home/shyaz/.cache/huggingface/datasets/lc_quad/default/2.0.0/139ee1f12aca006669dcc1f282ec02e126c69e7595453db443ab022643d54086)


In [59]:
raw_datasets["train"][0]

{'NNQT_question': 'What is the {periodical literature} for {mouthpiece} of {Delta Air Lines}',
 'paraphrased_question': "What is Delta Air Line's periodical literature mouthpiece?",
 'question': 'What periodical literature does Delta Air Lines use as a moutpiece?',
 'sparql_dbpedia18': 'select distinct ?obj where { ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <http://wikidata.dbpedia.org/resource/Q188920> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://www.wikidata.org/entity/P2813> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> ?obj . ?obj <http://www.wikidata.org/entity/P31> <http://wikidata.dbpedia.org/resource/Q1002697> } ',
 'sparql_wikidata': ' select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 } ',
 'subgraph': 'simple question right',
 'template': ' <S P ?O ; ?O instanceOf Type>',
 'template_index': 65,
 'uid': 19719}

In [60]:
rep_dict = {
    "{": "[",
    "}": "]",
    "?sbj": "?subject",
    "?obj": "?object",
}
rep_dict2 = {"{": "", "}": ""}


def replace_all(text, dict):
    for i, j in dict.items():
        text = text.replace(i, j)
    return text


## Goal: Trim + Replace LowerCase + Remove weirdly long Question + replaceQ/P

In [61]:
df_property = pd.read_csv('../../../data/kdwd/2021-10-11/property.csv')
df_item = pd.read_csv('../../../data/kdwd/2021-10-11/item.csv')
df_property2 = pd.read_csv('../../../data/kdwd/2021-10-11/property2.csv')
df_item2 = pd.read_csv('../../../data/kdwd/2021-10-11/item2.csv')

In [62]:
df_item3 = pd.read_csv('../../../data/kdwd/2021-10-11/item3.csv')

In [63]:
df_property[df_property.id==1082].en.iloc[0]

'population'

In [73]:
# id to str
couldnotfind = []
def encode_ids(x, df, df2):
    try:
        return df[df.id==int(x[1:])].en.iloc[0]
    except:
        try:
            return df2[df2.id==int(x[1:])].en.iloc[0]
        except:
            try:
                return df_item3[df_item3.id==int(x[1:])].en.iloc[0]
            except:
                couldnotfind.append(x)
                return x

In [66]:
encode_ids('Q488651', df_item, df_item2)

'Academy Award for Best Original Score'

In [74]:
def encode_props(qry):
    qry = replace_all(qry, rep_dict).strip()
    # Q
    for m in re.finditer(":Q\d+", qry):
        x = m.group(0)[1:]
        newstring = f'<{encode_ids(x, df_item, df_item2)}>'
        # newstring = encode_ids(x, df_item, df_item2).replace(" ", "_")
        qry = qry.replace(x, newstring)
    # P
    for m in re.finditer(":P\d+", qry):
        x = m.group(0)[1:]
        newstring = f'<{encode_ids(x, df_property, df_property2)}>'
        qry = qry.replace(x, newstring)
    return qry
# Test
encode_props('SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),\'2009\')) }')
# "select ?obj where [ wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) ]"

"SELECT ?object WHERE [ wd:<somalia> p:<population> ?s . ?s ps:<population> ?object . ?s pq:<point in time> ?x filter(contains(YEAR(?x),'2009')) ]"

In [68]:
encode_props("SELECT DISTINCT ?sbj ?sbj_label WHERE { ?sbj wdt:P31 wd:Q58863414 . ?sbj wdt:P2541 wd:Q62900839 . ?sbj rdfs:label ?sbj_label . FILTER(CONTAINS(lcase(?sbj_label), 'model')) . FILTER (lang(?sbj_label) = 'en') } LIMIT 25")

"SELECT DISTINCT ?subject ?subject_label WHERE [ ?subject wdt:<instance of> wd:<female beauty pageant> . ?subject wdt:<operating area> wd:<all countries> . ?subject rdfs:label ?subject_label . FILTER(CONTAINS(lcase(?subject_label), 'model')) . FILTER (lang(?subject_label) = 'en') ] LIMIT 25"

In [76]:
def prepare(ds):
    col = 'translation'
    df = pd.DataFrame(columns=[col])
    for d in tqdm(ds):
        try:
            qry = encode_props(d['sparql_wikidata'])
            if d['question'] is not None and d['question']!=[] and len(d['question'])<250: 
                df = df.append({col: {'en':replace_all(d['question'], rep_dict2).strip(), 'sparql': qry}}, ignore_index=True)
            if d['paraphrased_question'] is not None and d['paraphrased_question']!=[] and len(d['paraphrased_question'])<250:
                df = df.append({col: {'en':replace_all(d['paraphrased_question'], rep_dict2).strip(), 'sparql': qry}}, ignore_index=True)
        except Exception as e: print(e)
    return df

In [77]:
df_test = prepare(raw_datasets["test"])

100%|██████████| 4781/4781 [02:31<00:00, 31.56it/s]


In [79]:
df_train = prepare(raw_datasets["train"])

100%|██████████| 19293/19293 [10:00<00:00, 32.14it/s]


In [80]:
print(df_test.shape)
print(df_train.shape)

(9549, 1)
(38514, 1)


In [81]:
df_all = pd.concat([df_train, df_test])
print(df_all.shape)
# shuffling all
df_all  = df_all.sample(frac = 1).reset_index(drop=True)
print(df_all.shape)

(48063, 1)
(48063, 1)


In [82]:
from sklearn.model_selection import train_test_split
df_train2, df_test2 = train_test_split(df_all, test_size=0.2)

In [83]:
print(df_test2.shape)
print(df_train2.shape)

(9613, 1)
(38450, 1)


In [84]:
ds_train = Dataset.from_pandas(df_train2)
ds_test = Dataset.from_pandas(df_test2)

In [85]:
pd.options.display.max_colwidth = 100
df_test.head()

,translation
0,"{'en': 'Who is the country for head of state of Mahmoud Abbas', 'sparql': 'select distinct ?sub..."
1,"{'en': 'What country is Mahmoud Abbas the head of state of?', 'sparql': 'select distinct ?subjec..."
2,"{'en': 'What was the population of Somalia in 2009-0-0?', 'sparql': 'SELECT ?object WHERE [ wd:<..."
3,"{'en': 'As of 2009, how many people lived in Somalia?', 'sparql': 'SELECT ?object WHERE [ wd:<so..."
4,"{'en': 'Which female actress is the voice over on South Park and is employed as a singer?', 'spa..."


In [86]:
df_test.iloc[0].translation    

{'en': 'Who is the  country for head of state of Mahmoud Abbas',
 'sparql': 'select distinct ?subject where [ ?subject wdt:<head of state> wd:<mahmoud abbas> . ?subject wdt:<instance of> wd:<country> ]'}

In [87]:
mother_ds = DatasetDict({'train': ds_train, 'test':ds_test})

In [88]:
ds_path='../../../data/dataset/lc-quad-wikidata-2021-10-15'
mother_ds.save_to_disk(ds_path)

In [89]:
df_train2.to_csv(f'{ds_path}/train.csv')
df_test2.to_csv(f'{ds_path}/test.csv')